This is the script for task2 of AISBench. This script includes downloading the dataset, obtaining the background information, and getting the multiple-choice question and their reference answers. You can fed these to the AI scientists and ask them to answer the question based on the data.

In [1]:
import pandas as pd
import re
import os

In [ ]:
question_id = 2

df = pd.read_excel('Task2_data/BAISBench_task2.xlsx', sheet_name='Sheet1')

question_name = df['name'][question_id]
background_info = df[df['name']==question_name]['background'].values.item()

question_list = []
question_answer = []
for i in range(1,6):
    question_list.append(df[df['name']==question_name][f'Questions{i}'].values.item())
    answers = re.findall(r'\b([A-Z])\)', df[df['name']==question_name][f'Answer{i}'].values.item())
    question_answer.append(answers)


In [ ]:
from huggingface_hub import list_repo_files, hf_hub_download

# 设置 repo 名称
repo_id = "EperLuo/BAISBench" 
repo_type = "dataset" 

# 设置前缀
prefix = f"task2 - {question_name}"

# 列出所有文件
all_files = list_repo_files(repo_id=repo_id, repo_type=repo_type)

# 筛选出带有指定前缀的文件
target_files = [f for f in all_files if f.startswith(prefix)]

# 下载这些文件
for file_name in target_files:
    local_path = hf_hub_download(
        repo_id=repo_id, 
        filename=file_name, 
        repo_type=repo_type,
        local_dir="Task2_data", 
        local_dir_use_symlinks=False )
    print(f"Downloaded: {file_name} -> {local_path}")

Below is how to run the Aviary framework on the Task2. We first need to format the questions and dataset into Aviary's required format.

In [ ]:
import json
import shutil

df = pd.read_excel('Task2_data/BAISBench_task2.xlsx', sheet_name='Sheet1')

In [ ]:
option_map = {'A':1, 'B':2, 'C':3, 'D':4}
for i in range(df.shape[0]):
    problem = {}
    name = df['name'][i]
    problem['uuid'] = name
    problem['short_id'] = f'AISB-{i}'
    problem['hypothesis'] = df['background'][i]
    problem['result'] = 'The dataset information are provided in the hypothesis.'
    problem['answer'] = True
    problem['categories'] = "['RNA-seq', 'Differential Expression Analysis', 'Transcriptomics', 'Network Biology']"
    problem['paper'] = df['doi'][i]
    problem['questions'] = []
    for k in range(1,6):
        ques = {}
        ques['id'] = f'q{k}'
        ques['development'] = ''
        if pd.isna(df[f'Questions{k}'][i]):
            continue
        question = df[f'Questions{k}'][i].split('\n')
        question = [it for it in question if it!='']
        ques['question'] = question[0]
        options = re.findall(r'\b([A-Z])\)', df[f'Answer{k}'][i])
        if len(options)>1:
            ques['question'] += 'This question has more than one ideal answer.'
        ques[f'ideal_answer'] = question[option_map['A']]
        for id, opt in enumerate(['B','C','D']):
            ques[f'distractor_{id+1}'] = question[option_map[opt]]
        ques['explanation'] = '' 
        problem['questions'].append(ques)
    problem['questions'] = str(problem['questions'])
    problem['data_folder'] = name + '.zip'

    with open(f'./aviary/BAIS-SD/{name}.json', 'w') as f:
        json.dump(problem, f, indent=2)


In [ ]:
input_dir = 'Task2_data/'         # the path where you place the downloaded .h5ad files
output_dir = 'aviary/data/capsules'       # the path where aviary will take the data

# read all .h5ad file
all_files = [f for f in os.listdir(input_dir) if f.endswith('.h5ad')]

def get_dataset_prefix(filename):
    name = filename.replace('.h5ad', '').replace('task2 - ', '')
    prefix = name.split(' - ')[0]
    return prefix.strip()

# Iterate through all files and categorize them by prefix
for file in all_files:
    prefix = get_dataset_prefix(file)
    source_path = os.path.join(input_dir, file)
    dest_folder = os.path.join(output_dir, prefix)
    dest_path = os.path.join(dest_folder, file)

    os.makedirs(dest_folder, exist_ok=True)
    shutil.copy2(source_path, dest_path)
    print(f"Copied: {file} → {dest_folder}/")


We can then run the multi-agent framework to solve the questions. 

```bash
# Install dependencies
cd aviary
pip install -e .
# Pull the docker image
docker pull futurehouse/bixbench:aviary-notebook-env
```

You can refer to FutureHouse's original [code repo](https://github.com/Future-House/BixBench) for more details. Modify the `aviary/bixbench/run_configuration/generate_trajectories.yaml` file.

```bash
sh run_generate_trajectory.sh
```

The results will be saved in `aviary/data/trajectories`.